# Dheeraj J C - 1CR17CS036 (Prog 3)

Write a program to demonstrate the working of the decision tree based ID3 algorithm. Use an appropriate data set for building the decision tree and apply this knowledge to classify a new sample.

In [1]:
import pandas as pd
import math
from collections import Counter
from pprint import pprint
dataset=pd.read_csv('PlayTennis.csv')
dataset


,PlayTennis,Outlook,Temperature,Humidity,Windy
0,No,Sunny,Hot,High,Weak
1,No,Sunny,Hot,High,Strong
2,Yes,Overcast,Hot,High,Weak
3,Yes,Rain,Mild,High,Weak
4,Yes,Rain,Cool,Normal,Weak
5,No,Rain,Cool,Normal,Strong
6,Yes,Overcast,Cool,Normal,trong
7,No,Sunny,Mild,High,Weak
8,Yes,Sunny,Cool,Normal,Weak
9,Yes,Rain,Mild,Normal,Weak


In [2]:
def entropy(probs):
    
    return sum( [-prob*math.log(prob,2) for prob in probs])

def entropy_of_list(a_list):
    count=Counter(x for x in a_list)
    print("\nClasses:",count)
    num_instances=len(a_list)
    print("\n Number of Instances of the Current Sub Class is{0}:".format(num_instances))
    probs=[x / num_instances for x in count.values()]
    print(probs)
    print("\n Classes:",list(count.keys()))
    print("\n Probabilities of Class {0} is {1}:".format(min(count),min(probs)))
    print("\n Probabilities of Class{0} is {1}:".format(max(count),max(probs)))
    return entropy(probs)
print("\n INPUT DATA SET FOR ENTROPY CALCULATION:")
print(dataset['PlayTennis'])
total_entropy=entropy_of_list(dataset['PlayTennis'])
print("\n Total Entropy of PlayTennis Data Set:",total_entropy)


 INPUT DATA SET FOR ENTROPY CALCULATION:
0      No
1      No
2     Yes
3     Yes
4     Yes
5      No
6     Yes
7      No
8     Yes
9     Yes
10    Yes
11    Yes
12    Yes
13     No
Name: PlayTennis, dtype: object

Classes: Counter({'Yes': 9, 'No': 5})

 Number of Instances of the Current Sub Class is14:
[0.35714285714285715, 0.6428571428571429]

 Classes: ['No', 'Yes']

 Probabilities of Class No is 0.35714285714285715:

 Probabilities of ClassYes is 0.6428571428571429:

 Total Entropy of PlayTennis Data Set: 0.9402859586706309


In [3]:
def information_gain(df,split_attribute_name,target_attribute_name,trace=0):
    print("Information Gain Calculation of",split_attribute_name)
    df_split=df.groupby(split_attribute_name)
    print("split:",type(df_split))
    for name,group in df_split:
        print("Name:\n",name)
        print("Group:\n",group)
        
        nobs=len(df.index)
        print("NOBS",nobs)
        df_ent_prob=df_split.agg({target_attribute_name:[entropy_of_list,lambda x: len(x)/nobs]})[target_attribute_name]
        print(df_ent_prob.columns)
        print("the entropy and the probablity value for each attribute is",df_ent_prob)
        df_ent_prob.columns=['Entropy','PropObservations']
        new_entropy=sum(df_ent_prob['Entropy'] *df_ent_prob['PropObservations'])
        overall_entropy=entropy_of_list(df[target_attribute_name])
        return overall_entropy-new_entropy
    
print("Info-gain for Outlook is:"+str(information_gain(dataset,"Outlook","PlayTennis")),"\n")
print("\n Info-gain for the Humidity is"+str(information_gain(dataset,"Humidity","PlayTennis")),"\n")
print("\n Info-gain for Wind is"+str(information_gain(dataset,"Windy","PlayTennis")),"\n")
print("\n Info-gain for Temperature is"+str(information_gain(dataset,"Temperature","PlayTennis")),"\n")
        

Information Gain Calculation of Outlook
split: <class 'pandas.core.groupby.generic.DataFrameGroupBy'>
Name:
 Overcast
Group:
    PlayTennis   Outlook Temperature Humidity   Windy
2         Yes  Overcast         Hot     High    Weak
6         Yes  Overcast        Cool   Normal   trong
11        Yes  Overcast        Mild     High  Strong
12        Yes  Overcast         Hot   Normal    Weak
NOBS 14

Classes: Counter({'Yes': 4})

 Number of Instances of the Current Sub Class is4:
[1.0]

 Classes: ['Yes']

 Probabilities of Class Yes is 1.0:

 Probabilities of ClassYes is 1.0:

Classes: Counter({'Yes': 3, 'No': 2})

 Number of Instances of the Current Sub Class is5:
[0.6, 0.4]

 Classes: ['Yes', 'No']

 Probabilities of Class No is 0.4:

 Probabilities of ClassYes is 0.6:

Classes: Counter({'No': 3, 'Yes': 2})

 Number of Instances of the Current Sub Class is5:
[0.6, 0.4]

 Classes: ['No', 'Yes']

 Probabilities of Class No is 0.4:

 Probabilities of ClassYes is 0.6:
Index(['entropy_of_list

In [4]:
def id3(df,target_attribute_name,attribute_names,deafult_class=None):
    from collections import Counter
    count=Counter(x for x in df[target_attribute_name])
    if len(count)==1:
        return next(iter(count))
    elif df.empty or (not attribute_names):
        return defaukt_class
    else:
        default_class=max(count.keys())
        gainz=[information_gain(df,attr,target_attribute_name) for attr in attribute_names]
        index_of_max=gainz.index(max(gainz))
        best_attr=attribute_names[index_of_max]
        tree={best_attr:{}}
        remaining_attribute_names=[i for i in attribute_names if i != best_attr]
        for attr_val,data_subset in df.groupby(best_attr):
            subtree=id3(data_subset,
                       target_attribute_name,
                       remaining_attribute_names,
                       default_class)
            tree[best_attr][attr_val]=subtree
        return tree

In [5]:
attribute_names=list(dataset.columns)
print("List of Attributes:",attribute_names)
attribute_names.remove("PlayTennis")
print("Predicting Attributes:",attribute_names)

List of Attributes: ['PlayTennis', 'Outlook', 'Temperature', 'Humidity', 'Windy']
Predicting Attributes: ['Outlook', 'Temperature', 'Humidity', 'Windy']


In [6]:

tree=id3(dataset,"PlayTennis",attribute_names)
print("\n\n The Resultant Decision Tree is:\n")
pprint(tree)
attribute=next(iter(tree))
print("Best Attribute:\n",attribute)
print("Tree Keys:\n",tree[attribute].keys())

Information Gain Calculation of Outlook
split: <class 'pandas.core.groupby.generic.DataFrameGroupBy'>
Name:
 Overcast
Group:
    PlayTennis   Outlook Temperature Humidity   Windy
2         Yes  Overcast         Hot     High    Weak
6         Yes  Overcast        Cool   Normal   trong
11        Yes  Overcast        Mild     High  Strong
12        Yes  Overcast         Hot   Normal    Weak
NOBS 14

Classes: Counter({'Yes': 4})

 Number of Instances of the Current Sub Class is4:
[1.0]

 Classes: ['Yes']

 Probabilities of Class Yes is 1.0:

 Probabilities of ClassYes is 1.0:

Classes: Counter({'Yes': 3, 'No': 2})

 Number of Instances of the Current Sub Class is5:
[0.6, 0.4]

 Classes: ['Yes', 'No']

 Probabilities of Class No is 0.4:

 Probabilities of ClassYes is 0.6:

Classes: Counter({'No': 3, 'Yes': 2})

 Number of Instances of the Current Sub Class is5:
[0.6, 0.4]

 Classes: ['No', 'Yes']

 Probabilities of Class No is 0.4:

 Probabilities of ClassYes is 0.6:
Index(['entropy_of_list